# Notebook 11: Observability & Debugging

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 11 of 13  
**Level:** Advanced  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand the three pillars of observability (metrics, logs, traces)
2. Implement OpenTelemetry for MCP servers
3. Build distributed tracing across agent workflows
4. Create effective metrics and dashboards
5. Master MCP debugging techniques
6. Use MCP Inspector for development

## 1. Why Observability Matters for MCP

### The Agentic Complexity Challenge

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    OBSERVABILITY CHALLENGES IN AGENTIC AI                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   TRADITIONAL APP                      AGENTIC MCP SYSTEM               │
│   ═══════════════                      ══════════════════               │
│                                                                          │
│   Request ──► App ──► Response         User ──► Agent ──┬──► MCP A     │
│      │                  │                   │           ├──► MCP B     │
│      └── Single path ───┘                   │           └──► MCP C     │
│                                             │                │          │
│   • Predictable flow                        ▼                ▼          │
│   • Easy to trace                      Reasoning ──► More MCP calls    │
│   • Simple debugging                        │                │          │
│                                             ▼                ▼          │
│                                        Response (eventually)            │
│                                                                          │
│                                   • Non-deterministic paths             │
│                                   • Multiple concurrent calls           │
│                                   • Nested tool invocations             │
│                                   • LLM reasoning in between            │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   QUESTIONS OBSERVABILITY MUST ANSWER:                                  │
│                                                                          │
│   ❓ Why did the agent call this tool 5 times?                          │
│   ❓ Which MCP server is causing the latency?                           │
│   ❓ What was the actual data flow through the system?                  │
│   ❓ Why did the agent fail to complete the task?                       │
│   ❓ Are we hitting rate limits on any backend?                         │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### The Three Pillars of Observability

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    THREE PILLARS OF OBSERVABILITY                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐        │
│   │     METRICS     │  │      LOGS       │  │     TRACES      │        │
│   │                 │  │                 │  │                 │        │
│   │  📊 Numbers     │  │  📝 Events      │  │  🔗 Paths       │        │
│   │                 │  │                 │  │                 │        │
│   │  • Counters     │  │  • Structured   │  │  • Spans        │        │
│   │  • Gauges       │  │  • Timestamped  │  │  • Context      │        │
│   │  • Histograms   │  │  • Contextual   │  │  • Causality    │        │
│   │                 │  │                 │  │                 │        │
│   │  ANSWERS:       │  │  ANSWERS:       │  │  ANSWERS:       │        │
│   │  "How many?"    │  │  "What happened?"│ │  "How did we    │        │
│   │  "How fast?"    │  │  "Why failed?"  │  │   get here?"    │        │
│   │  "What's the    │  │  "What was the  │  │  "Where's the   │        │
│   │   trend?"       │  │   error?"       │  │   bottleneck?"  │        │
│   │                 │  │                 │  │                 │        │
│   └────────┬────────┘  └────────┬────────┘  └────────┬────────┘        │
│            │                    │                    │                  │
│            └────────────────────┼────────────────────┘                  │
│                                 │                                       │
│                                 ▼                                       │
│                    ┌─────────────────────┐                              │
│                    │   CORRELATION ID    │                              │
│                    │   Links all three   │                              │
│                    │   for a request     │                              │
│                    └─────────────────────┘                              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
import time
import random
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
import uuid
from contextlib import contextmanager

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. OpenTelemetry for MCP

### OpenTelemetry Overview

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    OPENTELEMETRY ARCHITECTURE                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   MCP SERVER                           OBSERVABILITY BACKENDS           │
│   ══════════                           ═════════════════════           │
│                                                                          │
│   ┌─────────────────┐                  ┌─────────────────┐              │
│   │  Application    │                  │   Prometheus    │              │
│   │  Code           │                  │   (Metrics)     │              │
│   └────────┬────────┘                  └─────────────────┘              │
│            │                                   ▲                        │
│            ▼                                   │                        │
│   ┌─────────────────┐                          │                        │
│   │  OpenTelemetry  │                  ┌───────┴───────┐                │
│   │  SDK            │                  │               │                │
│   │                 │     OTLP         │   OTel        │                │
│   │  • Tracer       │ ────────────────►│   Collector   │                │
│   │  • Meter        │                  │               │                │
│   │  • Logger       │                  └───────┬───────┘                │
│   └─────────────────┘                          │                        │
│                                        ┌───────┴───────┐                │
│                                        │               │                │
│                                        ▼               ▼                │
│                               ┌─────────────┐  ┌─────────────┐          │
│                               │   Jaeger    │  │   Loki      │          │
│                               │  (Traces)   │  │   (Logs)    │          │
│                               └─────────────┘  └─────────────┘          │
│                                                                          │
│   OTEL PROVIDES:                                                        │
│   • Vendor-neutral instrumentation                                      │
│   • Automatic context propagation                                       │
│   • Standardized data formats                                           │
│   • Language-specific SDKs                                              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# OpenTelemetry Setup for MCP Server
OTEL_SETUP_CODE = '''
# Install: pip install opentelemetry-api opentelemetry-sdk 
#          opentelemetry-exporter-otlp opentelemetry-instrumentation

from opentelemetry import trace, metrics
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.exporter.otlp.proto.grpc.metric_exporter import OTLPMetricExporter
from opentelemetry.sdk.resources import Resource, SERVICE_NAME

def setup_observability(service_name: str, otlp_endpoint: str = "localhost:4317"):
    """Initialize OpenTelemetry for an MCP server."""
    
    # Create resource identifying this service
    resource = Resource.create({
        SERVICE_NAME: service_name,
        "service.version": "1.0.0",
        "deployment.environment": "production"
    })
    
    # Setup tracing
    trace_provider = TracerProvider(resource=resource)
    trace_provider.add_span_processor(
        BatchSpanProcessor(OTLPSpanExporter(endpoint=otlp_endpoint))
    )
    trace.set_tracer_provider(trace_provider)
    
    # Setup metrics
    metric_reader = PeriodicExportingMetricReader(
        OTLPMetricExporter(endpoint=otlp_endpoint),
        export_interval_millis=60000  # Export every 60 seconds
    )
    meter_provider = MeterProvider(resource=resource, metric_readers=[metric_reader])
    metrics.set_meter_provider(meter_provider)
    
    return trace.get_tracer(service_name), metrics.get_meter(service_name)

# Usage
tracer, meter = setup_observability("my-mcp-server")
'''

print("OpenTelemetry Setup for MCP:")
print("=" * 60)
print(OTEL_SETUP_CODE)

## 3. Distributed Tracing for MCP

### Trace Propagation Across Services

In [ ]:
# Simplified Tracing Implementation (for demonstration)

@dataclass
class Span:
    """Represents a single unit of work in a trace."""
    trace_id: str
    span_id: str
    parent_span_id: Optional[str]
    operation_name: str
    service_name: str
    start_time: datetime
    end_time: Optional[datetime] = None
    status: str = "OK"
    attributes: Dict[str, Any] = field(default_factory=dict)
    events: List[Dict[str, Any]] = field(default_factory=list)
    
    @property
    def duration_ms(self) -> Optional[float]:
        if self.end_time:
            return (self.end_time - self.start_time).total_seconds() * 1000
        return None
    
    def add_event(self, name: str, attributes: Dict = None):
        self.events.append({
            "name": name,
            "timestamp": datetime.utcnow().isoformat(),
            "attributes": attributes or {}
        })
    
    def set_attribute(self, key: str, value: Any):
        self.attributes[key] = value
    
    def end(self, status: str = "OK"):
        self.end_time = datetime.utcnow()
        self.status = status
    
    def to_dict(self) -> Dict:
        return {
            "traceId": self.trace_id,
            "spanId": self.span_id,
            "parentSpanId": self.parent_span_id,
            "operationName": self.operation_name,
            "serviceName": self.service_name,
            "startTime": self.start_time.isoformat(),
            "endTime": self.end_time.isoformat() if self.end_time else None,
            "durationMs": self.duration_ms,
            "status": self.status,
            "attributes": self.attributes,
            "events": self.events
        }


class MCPTracer:
    """Tracer for MCP operations."""
    
    def __init__(self, service_name: str):
        self.service_name = service_name
        self.spans: List[Span] = []
        self._current_span: Optional[Span] = None
    
    def _generate_id(self) -> str:
        return uuid.uuid4().hex[:16]
    
    @contextmanager
    def start_span(self, operation_name: str, parent_span: Span = None):
        """Start a new span."""
        trace_id = parent_span.trace_id if parent_span else self._generate_id()
        parent_id = parent_span.span_id if parent_span else None
        
        span = Span(
            trace_id=trace_id,
            span_id=self._generate_id(),
            parent_span_id=parent_id,
            operation_name=operation_name,
            service_name=self.service_name,
            start_time=datetime.utcnow()
        )
        
        previous_span = self._current_span
        self._current_span = span
        
        try:
            yield span
            span.end("OK")
        except Exception as e:
            span.set_attribute("error", True)
            span.set_attribute("error.message", str(e))
            span.end("ERROR")
            raise
        finally:
            self.spans.append(span)
            self._current_span = previous_span
    
    def get_current_span(self) -> Optional[Span]:
        return self._current_span


# MCP-specific tracing decorators
def trace_tool_call(tracer: MCPTracer):
    """Decorator to trace MCP tool calls."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            with tracer.start_span(f"tool/{func.__name__}") as span:
                span.set_attribute("mcp.tool.name", func.__name__)
                span.set_attribute("mcp.tool.arguments", str(kwargs)[:200])
                
                span.add_event("tool_call_started")
                result = func(*args, **kwargs)
                span.add_event("tool_call_completed")
                
                span.set_attribute("mcp.tool.success", True)
                return result
        return wrapper
    return decorator


# Demo
tracer = MCPTracer("github-mcp")

# Simulate a traced operation
with tracer.start_span("handle_request") as root_span:
    root_span.set_attribute("mcp.method", "tools/call")
    root_span.set_attribute("mcp.request_id", "req-12345")
    
    # Simulate tool execution
    with tracer.start_span("tool/create_issue", parent_span=root_span) as tool_span:
        tool_span.set_attribute("github.repo", "owner/repo")
        tool_span.add_event("github_api_called")
        time.sleep(0.1)  # Simulate work
        tool_span.set_attribute("github.issue_number", 42)

print("Distributed Tracing Demo:")
print("=" * 60)
print(f"\nTraced {len(tracer.spans)} spans:")
for span in tracer.spans:
    indent = "  " if span.parent_span_id else ""
    print(f"{indent}• {span.operation_name} ({span.duration_ms:.1f}ms) - {span.status}")

print("\nFull trace data:")
pprint(tracer.spans[0].to_dict())

### Visualizing MCP Traces

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP TRACE VISUALIZATION (JAEGER-STYLE)                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  Trace: abc123   Duration: 450ms   Services: 3   Spans: 7               │
│                                                                          │
│  ├─ agent-orchestrator ─────────────────────────────────────────────┤   │
│  │  handle_user_request (450ms)                                     │   │
│  │                                                                   │   │
│  │  ├─ mcp-gateway ───────────────────────────────────┤             │   │
│  │  │  route_request (420ms)                          │             │   │
│  │  │                                                  │             │   │
│  │  │  ├─ github-mcp ────────────────────┤            │             │   │
│  │  │  │  tool/search_code (150ms)       │            │             │   │
│  │  │  │  └─ github_api_call (120ms)     │            │             │   │
│  │  │  │                                  │            │             │   │
│  │  │  ├─ database-mcp ──────────────────┤            │             │   │
│  │  │  │  tool/query (200ms)             │            │             │   │
│  │  │  │  └─ postgres_query (180ms)      │            │             │   │
│  │  │  │                                  │            │             │   │
│  │  │  └─ github-mcp ──────┤              │            │             │   │
│  │  │     tool/create_issue (50ms)       │            │             │   │
│  │  │                                     │            │             │   │
│                                                                          │
│  Timeline: 0ms ─────────────────────────────────────────────── 450ms    │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 4. Metrics for MCP Servers

### Key Metrics to Track

In [ ]:
# MCP Metrics Implementation

class MetricType(Enum):
    COUNTER = "counter"
    GAUGE = "gauge"
    HISTOGRAM = "histogram"


@dataclass
class Metric:
    """A single metric."""
    name: str
    type: MetricType
    description: str
    labels: Dict[str, str] = field(default_factory=dict)
    value: float = 0
    histogram_buckets: List[float] = field(default_factory=list)
    histogram_counts: List[int] = field(default_factory=list)


class MCPMetrics:
    """Metrics collector for MCP servers."""
    
    def __init__(self, service_name: str):
        self.service_name = service_name
        self.metrics: Dict[str, Metric] = {}
        self._setup_default_metrics()
    
    def _setup_default_metrics(self):
        """Initialize standard MCP metrics."""
        # Request metrics
        self._create_counter(
            "mcp_requests_total",
            "Total number of MCP requests"
        )
        self._create_counter(
            "mcp_requests_failed_total",
            "Total number of failed MCP requests"
        )
        
        # Tool metrics
        self._create_counter(
            "mcp_tool_calls_total",
            "Total number of tool calls"
        )
        self._create_histogram(
            "mcp_tool_duration_seconds",
            "Tool call duration in seconds",
            buckets=[0.01, 0.05, 0.1, 0.25, 0.5, 1, 2.5, 5, 10]
        )
        
        # Resource metrics
        self._create_counter(
            "mcp_resource_reads_total",
            "Total number of resource reads"
        )
        
        # Connection metrics
        self._create_gauge(
            "mcp_active_connections",
            "Number of active MCP connections"
        )
    
    def _create_counter(self, name: str, description: str):
        self.metrics[name] = Metric(
            name=name,
            type=MetricType.COUNTER,
            description=description
        )
    
    def _create_gauge(self, name: str, description: str):
        self.metrics[name] = Metric(
            name=name,
            type=MetricType.GAUGE,
            description=description
        )
    
    def _create_histogram(self, name: str, description: str, buckets: List[float]):
        self.metrics[name] = Metric(
            name=name,
            type=MetricType.HISTOGRAM,
            description=description,
            histogram_buckets=buckets,
            histogram_counts=[0] * (len(buckets) + 1)
        )
    
    def increment(self, name: str, value: float = 1, labels: Dict = None):
        """Increment a counter."""
        key = self._metric_key(name, labels)
        if key not in self.metrics:
            self.metrics[key] = Metric(
                name=name,
                type=MetricType.COUNTER,
                description="",
                labels=labels or {}
            )
        self.metrics[key].value += value
    
    def set_gauge(self, name: str, value: float, labels: Dict = None):
        """Set a gauge value."""
        key = self._metric_key(name, labels)
        if key not in self.metrics:
            self.metrics[key] = Metric(
                name=name,
                type=MetricType.GAUGE,
                description="",
                labels=labels or {}
            )
        self.metrics[key].value = value
    
    def observe(self, name: str, value: float, labels: Dict = None):
        """Record a histogram observation."""
        key = self._metric_key(name, labels)
        metric = self.metrics.get(key)
        if metric and metric.type == MetricType.HISTOGRAM:
            # Find bucket and increment
            for i, bucket in enumerate(metric.histogram_buckets):
                if value <= bucket:
                    metric.histogram_counts[i] += 1
                    break
            else:
                metric.histogram_counts[-1] += 1  # +Inf bucket
    
    def _metric_key(self, name: str, labels: Dict = None) -> str:
        """Create unique key for metric with labels."""
        if not labels:
            return name
        label_str = ",".join(f"{k}={v}" for k, v in sorted(labels.items()))
        return f"{name}{{{label_str}}}"
    
    def record_tool_call(self, tool_name: str, duration_seconds: float, success: bool):
        """Record metrics for a tool call."""
        labels = {"tool": tool_name}
        
        self.increment("mcp_tool_calls_total", labels=labels)
        self.observe("mcp_tool_duration_seconds", duration_seconds, labels=labels)
        
        if not success:
            self.increment("mcp_requests_failed_total", labels=labels)
    
    def get_prometheus_format(self) -> str:
        """Export metrics in Prometheus format."""
        lines = []
        
        for key, metric in self.metrics.items():
            lines.append(f"# HELP {metric.name} {metric.description}")
            lines.append(f"# TYPE {metric.name} {metric.type.value}")
            
            if metric.labels:
                label_str = ",".join(f'{k}="{v}"' for k, v in metric.labels.items())
                lines.append(f"{metric.name}{{{label_str}}} {metric.value}")
            else:
                lines.append(f"{metric.name} {metric.value}")
        
        return "\n".join(lines)


# Demo
metrics = MCPMetrics("github-mcp")

# Simulate tool calls
tools = ["create_issue", "search_code", "list_repos", "create_issue"]
for tool in tools:
    duration = random.uniform(0.05, 0.5)
    success = random.random() > 0.1  # 90% success rate
    metrics.record_tool_call(tool, duration, success)

# Set connection gauge
metrics.set_gauge("mcp_active_connections", 5)

print("MCP Metrics Demo:")
print("=" * 60)
print("\nPrometheus Format:")
print(metrics.get_prometheus_format())

### Essential MCP Metrics Dashboard

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP SERVER DASHBOARD                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌─────────────────────┐  ┌─────────────────────┐  ┌─────────────────┐  │
│  │   Request Rate      │  │   Error Rate        │  │  Active Conns   │  │
│  │   ▄▄▆█▇▅▄▃▄▅▆▇█     │  │   ▁▁▂▁▁▃▁▁▁▁▂▁     │  │      42         │  │
│  │   1,234 req/min     │  │     2.1%            │  │                 │  │
│  └─────────────────────┘  └─────────────────────┘  └─────────────────┘  │
│                                                                          │
│  ┌────────────────────────────────────────────────────────────────────┐ │
│  │  Tool Call Latency (p50, p95, p99)                                 │ │
│  │                                                                     │ │
│  │  800ms ┤                                                            │ │
│  │  600ms ┤                    ╭─────────╮                            │ │
│  │  400ms ┤               ╭────╯         ╰────╮        p99            │ │
│  │  200ms ┤          ╭────╯                    ╰───╮    p95            │ │
│  │  100ms ┤     ╭────╯                              ╰── p50            │ │
│  │        └─────────────────────────────────────────────────────────── │ │
│  │             09:00    10:00    11:00    12:00    13:00              │ │
│  └────────────────────────────────────────────────────────────────────┘ │
│                                                                          │
│  ┌─────────────────────────────────┐  ┌─────────────────────────────┐   │
│  │  Top Tools by Call Volume       │  │  Top Tools by Error Rate    │   │
│  │                                 │  │                             │   │
│  │  search_code     ████████ 45%   │  │  delete_file    █████ 8.2%  │   │
│  │  create_issue    █████ 25%      │  │  query_db       ██ 3.1%     │   │
│  │  list_repos      ████ 20%       │  │  create_issue   █ 1.5%      │   │
│  │  get_file        ██ 10%         │  │  search_code    ▏ 0.2%      │   │
│  │                                 │  │                             │   │
│  └─────────────────────────────────┘  └─────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 5. Structured Logging for MCP

### JSON Logging Best Practices

In [ ]:
# Structured Logging for MCP
import logging
import sys

class MCPJSONFormatter(logging.Formatter):
    """JSON formatter for MCP logs."""
    
    def format(self, record: logging.LogRecord) -> str:
        log_entry = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "level": record.levelname,
            "logger": record.name,
            "message": record.getMessage(),
            "service": getattr(record, "service", "mcp-server"),
        }
        
        # Add trace context if available
        if hasattr(record, "trace_id"):
            log_entry["trace_id"] = record.trace_id
        if hasattr(record, "span_id"):
            log_entry["span_id"] = record.span_id
        
        # Add MCP-specific context
        if hasattr(record, "mcp_method"):
            log_entry["mcp"] = {
                "method": record.mcp_method,
                "tool": getattr(record, "mcp_tool", None),
                "request_id": getattr(record, "request_id", None)
            }
        
        # Add error details
        if record.exc_info:
            log_entry["error"] = {
                "type": record.exc_info[0].__name__ if record.exc_info[0] else None,
                "message": str(record.exc_info[1]) if record.exc_info[1] else None
            }
        
        # Add extra fields
        for key, value in record.__dict__.items():
            if key.startswith("extra_"):
                log_entry[key[6:]] = value  # Remove "extra_" prefix
        
        return json.dumps(log_entry)


class MCPLogger:
    """Logger with MCP context support."""
    
    def __init__(self, name: str, service: str):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        self.service = service
        
        # Note: In real MCP server, log to stderr
        # handler = logging.StreamHandler(sys.stderr)
        # For demo, we'll collect logs
        self.logs = []
    
    def _log(self, level: int, message: str, **kwargs):
        """Internal log method."""
        log_entry = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "level": logging.getLevelName(level),
            "service": self.service,
            "message": message,
            **kwargs
        }
        self.logs.append(log_entry)
    
    def info(self, message: str, **kwargs):
        self._log(logging.INFO, message, **kwargs)
    
    def warning(self, message: str, **kwargs):
        self._log(logging.WARNING, message, **kwargs)
    
    def error(self, message: str, **kwargs):
        self._log(logging.ERROR, message, **kwargs)
    
    def debug(self, message: str, **kwargs):
        self._log(logging.DEBUG, message, **kwargs)
    
    def tool_call_started(self, tool_name: str, request_id: str, trace_id: str = None):
        """Log tool call start."""
        self.info(
            f"Tool call started: {tool_name}",
            mcp_method="tools/call",
            mcp_tool=tool_name,
            request_id=request_id,
            trace_id=trace_id,
            event="tool_call_started"
        )
    
    def tool_call_completed(self, tool_name: str, request_id: str, duration_ms: float, success: bool):
        """Log tool call completion."""
        level = "info" if success else "error"
        getattr(self, level)(
            f"Tool call completed: {tool_name}",
            mcp_method="tools/call",
            mcp_tool=tool_name,
            request_id=request_id,
            duration_ms=duration_ms,
            success=success,
            event="tool_call_completed"
        )


# Demo
logger = MCPLogger("github-mcp", "github-mcp-server")

# Simulate logging
request_id = "req-" + uuid.uuid4().hex[:8]
trace_id = "trace-" + uuid.uuid4().hex[:16]

logger.info("Server started", version="1.0.0", port=8000)
logger.tool_call_started("create_issue", request_id, trace_id)
logger.debug("Calling GitHub API", endpoint="/repos/owner/repo/issues")
logger.tool_call_completed("create_issue", request_id, 150.5, True)
logger.warning("Rate limit approaching", remaining=10, limit=100)

print("Structured Logging Demo:")
print("=" * 60)
for log in logger.logs:
    print(json.dumps(log, indent=2))
    print()

## 6. MCP Inspector for Debugging

### Using MCP Inspector

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP INSPECTOR                                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   INSTALLATION:                                                         │
│   npm install -g @modelcontextprotocol/inspector                        │
│                                                                          │
│   USAGE:                                                                │
│   npx @modelcontextprotocol/inspector python your_server.py             │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │  MCP Inspector - http://localhost:5173                          │   │
│   ├─────────────────────────────────────────────────────────────────┤   │
│   │                                                                  │   │
│   │  Server: github-mcp (connected)              [Refresh] [Clear]  │   │
│   │                                                                  │   │
│   │  ┌──────────────────────────────────────────────────────────┐   │   │
│   │  │  TOOLS (5)                                               │   │   │
│   │  │  ├─ create_issue                                         │   │   │
│   │  │  ├─ search_code                                          │   │   │
│   │  │  ├─ list_repos                                           │   │   │
│   │  │  ├─ get_file                                             │   │   │
│   │  │  └─ create_pr                                            │   │   │
│   │  └──────────────────────────────────────────────────────────┘   │   │
│   │                                                                  │   │
│   │  ┌──────────────────────────────────────────────────────────┐   │   │
│   │  │  TEST TOOL: create_issue                                 │   │   │
│   │  │                                                          │   │   │
│   │  │  Arguments:                                              │   │   │
│   │  │  ┌────────────────────────────────────────────────────┐  │   │   │
│   │  │  │ {                                                  │  │   │   │
│   │  │  │   "repo": "owner/repo",                            │  │   │   │
│   │  │  │   "title": "Test issue",                           │  │   │   │
│   │  │  │   "body": "Created via MCP Inspector"              │  │   │   │
│   │  │  │ }                                                  │  │   │   │
│   │  │  └────────────────────────────────────────────────────┘  │   │   │
│   │  │                                           [Execute]      │   │   │
│   │  └──────────────────────────────────────────────────────────┘   │   │
│   │                                                                  │   │
│   │  ┌──────────────────────────────────────────────────────────┐   │   │
│   │  │  MESSAGE LOG                                             │   │   │
│   │  │  ─────────────────────────────────────────────────────── │   │   │
│   │  │  → tools/call create_issue {...}              12:34:56   │   │   │
│   │  │  ← result {"issue_number": 42}                12:34:57   │   │   │
│   │  │  → tools/list                                 12:34:50   │   │   │
│   │  │  ← result {"tools": [...]}                    12:34:50   │   │   │
│   │  └──────────────────────────────────────────────────────────┘   │   │
│   │                                                                  │   │
│   └──────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# MCP Inspector Commands
inspector_commands = {
    "start_inspector": {
        "description": "Start MCP Inspector with a server",
        "commands": [
            "# For Python servers",
            "npx @modelcontextprotocol/inspector python your_server.py",
            "",
            "# For Node.js servers",
            "npx @modelcontextprotocol/inspector node your_server.js",
            "",
            "# With environment variables",
            "npx @modelcontextprotocol/inspector -e API_KEY=xxx python server.py",
            "",
            "# With arguments",
            "npx @modelcontextprotocol/inspector python server.py -- --config config.json"
        ]
    },
    "features": [
        "Interactive tool testing with JSON editor",
        "Real-time message log (requests/responses)",
        "Resource browser and viewer",
        "Prompt template testing",
        "Server capability inspection",
        "Export/import test cases"
    ]
}

print("MCP Inspector Usage:")
print("=" * 60)
print("\nCommands:")
for cmd in inspector_commands["start_inspector"]["commands"]:
    print(f"  {cmd}")
print("\nFeatures:")
for feature in inspector_commands["features"]:
    print(f"  • {feature}")

## 7. Common Debugging Scenarios

### Debugging Checklist

In [ ]:
# Common MCP Issues and Solutions
debugging_guide = {
    "connection_issues": {
        "symptoms": [
            "Server not showing in Claude Desktop",
            "Connection timeout",
            "'Server disconnected' errors"
        ],
        "checks": [
            "Verify server path in claude_desktop_config.json is absolute",
            "Check Python/Node is in PATH",
            "Look for errors in Claude Desktop logs",
            "Test server directly with MCP Inspector",
            "Ensure PYTHONUNBUFFERED=1 is set for Python servers"
        ],
        "log_location": {
            "windows": "%APPDATA%\\Claude\\logs\\",
            "macos": "~/Library/Logs/Claude/",
            "linux": "~/.local/share/Claude/logs/"
        }
    },
    "tool_not_working": {
        "symptoms": [
            "Tool returns error",
            "Tool not appearing in list",
            "Arguments validation failing"
        ],
        "checks": [
            "Verify JSON Schema is valid",
            "Check tool name matches exactly",
            "Test with MCP Inspector first",
            "Look for Python exceptions in stderr",
            "Verify all required fields have values"
        ]
    },
    "slow_performance": {
        "symptoms": [
            "Tools taking too long",
            "Timeouts on complex operations",
            "Memory usage growing"
        ],
        "checks": [
            "Add timing logs to identify bottleneck",
            "Check for N+1 query patterns",
            "Profile with cProfile or py-spy",
            "Consider async operations for I/O",
            "Check if external APIs are slow"
        ]
    },
    "stdout_corruption": {
        "symptoms": [
            "Server crashes randomly",
            "Invalid JSON-RPC errors",
            "Partial responses"
        ],
        "checks": [
            "NEVER use print() - it goes to stdout",
            "Always log to stderr: logging.basicConfig(stream=sys.stderr)",
            "Check libraries aren't printing to stdout",
            "Verify JSON responses are complete"
        ]
    }
}

print("MCP Debugging Guide:")
print("=" * 60)

for issue, details in debugging_guide.items():
    print(f"\n🔍 {issue.upper().replace('_', ' ')}")
    print("-" * 40)
    print("Symptoms:")
    for s in details["symptoms"]:
        print(f"  ⚠️  {s}")
    print("Checks:")
    for c in details["checks"]:
        print(f"  ✓ {c}")

## 8. Observability Stack Setup

### Docker Compose for Local Development

In [ ]:
# Docker Compose for Observability Stack
docker_compose = '''
version: '3.8'

services:
  # OpenTelemetry Collector
  otel-collector:
    image: otel/opentelemetry-collector-contrib:latest
    command: ["--config=/etc/otel-collector-config.yaml"]
    volumes:
      - ./otel-collector-config.yaml:/etc/otel-collector-config.yaml
    ports:
      - "4317:4317"   # OTLP gRPC
      - "4318:4318"   # OTLP HTTP
      - "8889:8889"   # Prometheus metrics
    depends_on:
      - jaeger
      - prometheus

  # Jaeger for Traces
  jaeger:
    image: jaegertracing/all-in-one:latest
    ports:
      - "16686:16686"  # UI
      - "14268:14268"  # Collector
    environment:
      - COLLECTOR_OTLP_ENABLED=true

  # Prometheus for Metrics
  prometheus:
    image: prom/prometheus:latest
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
    ports:
      - "9090:9090"
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'

  # Grafana for Dashboards
  grafana:
    image: grafana/grafana:latest
    ports:
      - "3000:3000"
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
    volumes:
      - grafana-storage:/var/lib/grafana
      - ./grafana/provisioning:/etc/grafana/provisioning
    depends_on:
      - prometheus
      - jaeger

  # Loki for Logs
  loki:
    image: grafana/loki:latest
    ports:
      - "3100:3100"
    command: -config.file=/etc/loki/local-config.yaml

volumes:
  grafana-storage:
'''

print("Docker Compose for Observability Stack:")
print("=" * 60)
print(docker_compose)
print("\n📁 Save as: docker-compose.observability.yml")
print("▶️  Run: docker-compose -f docker-compose.observability.yml up -d")

## 9. Key Takeaways

### Observability Best Practices Summary

| Pillar | Tool | Key Metrics/Data |
|--------|------|------------------|
| **Traces** | Jaeger/Zipkin | Request flow, latency breakdown, error paths |
| **Metrics** | Prometheus | Request rate, error rate, latency percentiles |
| **Logs** | Loki/ELK | Structured JSON, correlation IDs, error details |

### Critical MCP Observability Rules

1. **Always use correlation IDs** - Link traces, metrics, and logs
2. **Log to stderr** - Never corrupt stdout JSON-RPC stream
3. **Include MCP context** - Tool name, request ID, user in every log
4. **Track tool-level metrics** - Different tools have different profiles
5. **Use MCP Inspector** - Test before integrating with Claude

### Next Notebook

In **Notebook 12: Framework Integration**, we'll cover:
- LangChain MCP integration
- LangGraph with MCP tools
- CrewAI multi-agent MCP
- Custom framework integration

## 10. Exercises

### Exercise 1: Add Tracing to MCP Server

Add distributed tracing to this MCP tool:

In [ ]:
# Exercise 1: Add tracing to this tool

def search_database(query: str, limit: int = 10) -> List[Dict]:
    """Search database for matching records.
    
    TODO: Add tracing with:
    - Span for the overall search
    - Child span for query execution
    - Attributes for query and limit
    - Event for results count
    """
    # Simulate database search
    time.sleep(0.1)
    results = [{"id": i, "match": f"Result {i}"} for i in range(min(limit, 5))]
    return results

# Your traced version here:
# def search_database_traced(tracer: MCPTracer, query: str, limit: int = 10):
#     ...

### Exercise 2: Create Custom Metrics

Add metrics for a rate-limited API tool:

In [ ]:
# Exercise 2: Add metrics for rate limiting

class RateLimitedAPIMetrics:
    """Metrics for a rate-limited external API.
    
    TODO: Track:
    - API calls total (by endpoint)
    - Rate limit remaining (gauge)
    - Rate limit hits (counter)
    - API response time (histogram)
    """
    
    def __init__(self, api_name: str):
        self.api_name = api_name
        # Your metrics setup here
    
    def record_call(self, endpoint: str, duration: float, rate_limit_remaining: int):
        """Record an API call."""
        # Your implementation here
        pass
    
    def record_rate_limit_hit(self, endpoint: str):
        """Record a rate limit hit."""
        # Your implementation here
        pass

### Exercise 3: Build Debug Dashboard Query

Write a log query to find slow tool calls:

In [ ]:
# Exercise 3: Log query for debugging

def find_slow_tool_calls(logs: List[Dict], threshold_ms: float = 1000) -> List[Dict]:
    """Find tool calls that exceeded the latency threshold.
    
    Args:
        logs: List of structured log entries
        threshold_ms: Latency threshold in milliseconds
    
    Returns:
        List of slow tool call logs with:
        - tool name
        - duration
        - timestamp
        - trace_id (for further investigation)
    """
    # Your implementation here
    pass

# Test with sample logs
sample_logs = [
    {"event": "tool_call_completed", "mcp_tool": "search", "duration_ms": 150, "trace_id": "abc"},
    {"event": "tool_call_completed", "mcp_tool": "query", "duration_ms": 2500, "trace_id": "def"},
    {"event": "tool_call_completed", "mcp_tool": "fetch", "duration_ms": 800, "trace_id": "ghi"},
]

# slow = find_slow_tool_calls(sample_logs, threshold_ms=1000)
# print(f"Found {len(slow)} slow calls")

---

## References

- [OpenTelemetry Python](https://opentelemetry.io/docs/instrumentation/python/)
- [MCP Inspector](https://github.com/modelcontextprotocol/inspector)
- [Jaeger Tracing](https://www.jaegertracing.io/docs/)
- [Prometheus Metrics](https://prometheus.io/docs/introduction/overview/)
- [Grafana Dashboards](https://grafana.com/docs/grafana/latest/)

---

**Next Notebook:** [12_Framework_Integration.ipynb](./12_Framework_Integration.ipynb)